In [1]:
import json
from konlpy.tag import Kkma
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC,SVR
from sklearn.metrics import accuracy_score,mean_squared_error,roc_auc_score
import numpy as np
from sklearn.linear_model import LogisticRegression,SGDClassifier

In [2]:
REVIEWS = "cs_reviews"
POS = "pos"

In [3]:
f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

In [4]:
g = open("../reviews_in_one_txt/" + REVIEWS + "_" + POS + ".txt","r",encoding="utf-8").readlines()

In [5]:
# kkma = Kkma()
# with open("../reviews_in_one_txt/" + REVIEWS+"_nouns.txt","w",encoding="utf-8") as out:
#     for line in f:
#         course = json.loads(line)
#         nouns = kkma.nouns(course["review_text"])
# #         print(nouns)
#         out.write(" ".join(nouns)+"\n")
        

In [194]:
count = {}
for i in range(len(f)):
    course = json.loads(f[i])
    if course["강의"] != "?" and len(g[i])>1:
        if course["강의"] not in count:
            count[course["강의"]] = 0
        count[course["강의"]] += 1
        
print(count)
upper_limit = min(count.values())
upper_limit = 100
print(upper_limit)

{'F': 37, 'A': 263, 'C': 62, 'B': 98, 'D': 38}
100


In [195]:
concat_nouns_list = []
for k,v in count.items():
    count[k] = 0
# mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
mp = {"A":1,"B":1,"C":0,"D":0,"F":0}

labels = []
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] != "?" and len(g[i])>1:
        concat_nouns = g[i]
        if count[course["강의"]] < upper_limit:
            concat_nouns_list.append(concat_nouns)
            labels.append(int(mp[course["강의"]]))
            count[course["강의"]] += 1

In [196]:
labels = np.array(labels)
concat_nouns_list = np.array(concat_nouns_list)
data_train, data_test, y_train, y_test = train_test_split(concat_nouns_list,
                                    labels,test_size=0.1,random_state=42)

In [197]:
len(data_train)

301

In [198]:
len(y_train)

301

In [199]:
# list(data_train)[:2]

In [200]:
y_train[:2]

array([0, 1])

In [201]:
def parse_long_str(s):
    s = s.split(',')
    s = [x.strip() for x in s]
    return s
good_lecture = parse_long_str("사랑, 천사, 친절, 유명, 짱, 참여, 최고, 만족, 유익, 열정적, 갓, 답변, 명강의")
bad_lecture = parse_long_str("집중, 혼자, 부족, 자습, 당황, 최악, 인생, 불편, 복잡, 불만, 악명, 비추, 난이(도)")
#todo try weighted average of w2v
scores = {}
def add_to_dict(l,score):
    global scores
    print("Added " + str(len(l)))
    for i in range(len(l)):
        scores[l[i]] = score
add_to_dict(good_lecture,1.0)
add_to_dict(bad_lecture,-1.0)

Added 13
Added 13


In [202]:
def enlarge_train_data(data_train,y_train):
    x = list(data_train)
    y = list(y_train)
    for k,v in scores.items():
#         x.append(str(k)+"\n")
        x.append("(%s,NNG)" % (str(k))+"\n")
        if v > 0:
            y.append(int(mp['A']))
        else:
            y.append(int(mp['F']))
    
    return np.array(x),np.array(y)

# data_train,y_train = enlarge_train_data(data_train,y_train)

In [203]:
len(data_train)

301

In [204]:
len(y_train)

301

In [214]:
vectorizer = TfidfVectorizer(min_df=3, max_df = 0.8,max_features=200000, ngram_range=(3,5))

In [215]:
# vectirizer.fit(..)
# X_train = vectorizer.transform(data_train)
X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)

In [216]:
print(X_train.shape)

(301, 9412)


In [217]:
print(X_test.shape)

(34, 9412)


In [218]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [219]:
predictions = clf.predict(X_test)
accuracy_score(y_test,predictions)

0.7352941176470589

In [220]:
mean_squared_error(y_test,predictions)

0.2647058823529412

In [221]:
predictions

array([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [222]:
y_test

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0])

In [146]:
for i in range(len(predictions)):
    if predictions[i]!= y_test[i]:
        print("pred",predictions[i],"real",y_test[i])
        print(data_test[i])
        

pred 3 real 5
(전산,NNG) (과,JC) (전공,NNG) (필수,NNG) (,,SP) (데이타,NNG) (구조,NNG) (이,VCP) (ㅂ니다,EFN) (.,SF) (아마,MAG) (전산,NNG) (과,JKM) (진입,NNG) (생,XSN) (여러분,NP) (이,JKS) (가장,MAG) (처음,NNG) (으로,JKM) (마주치,VV) (는,ETD) (전공,NNG) (과목,NNG) (이,VCP) (ㄹ,ETD) (것,NNB) (같,VA) (네요,EFN) (.,SF) (언어,NNG) (는,JX) (Java,OL) (를,JKO) (쓰,VV) (고,ECE) (,,SP) (elice,OL) (.,SF) (io,OL) (라,NNG) (는,JX) (플랫폼,NNG) (에서,JKM) (수업,NNG) (을,JKO) (하,VV) (ㅂ니다,EFN) (.,SF) (퀴즈,NNG) (10,NR) (번,NNM) (,,SP) (과제,NNG) (6,NR) (번,NNM) (,,SP) ((,SS) (과제,NNG) (0,NR) (포함,NNG) (),SS) (미,XPN) (니플,NNG) (젝,UN) (두,MDN) (번,NNB) (이,JKS) (있,VV) (었,EPT) (고,ECE) (,,SP) (중간,NNG) (,,SP) (기말,NNG) (이,JKS) (있,VV) (었,EPT) (습니다,EFN) (.,SF) (과제,NNG) (는,JX) (난이도,NNG) (가,JKS) (그렇게,MAG) (어렵,VA) (ㄴ,ETD) (편,NNB) (은,JX) (아니,VCN) (지만,ECE) (,,SP) (하나,NR) (의,JKG) (과제,NNG) (에,JKM) (5,NR) (문제,NNG) (정도,NNG) (가,JKS) (있,VV) (어서,ECD) (하,VV) (는,ETD) (데,NNB) (시간,NNG) (은,JX) (꽤,MAG) (걸리,VV) (ㅂ니다,EFN) (.,SF) (수업,NNG) (은,JX) (영어,NNG) (수업,NNG) (이,VCP) (ㄴ데,ECE) (영어,NNG) (가,JKS) (유창,XR) 